In [0]:
!pip install tflearn
!pip install pydrive
!pip install matplotlib


In [0]:
from __future__ import division, print_function, absolute_import

import pickle
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [0]:
from pydrive.drive import GoogleDrive

drive = GoogleDrive(gauth) # Create GoogleDrive instance with authenticated GoogleAuth instance

# Auto-iterate through all files in the root folder.
file_list = drive.ListFile({'q': "'1vQhQXjy4X9woILOTXeY3C-tz14xJkBQE' in parents and trashed=false"}).GetList()
for file1 in file_list:
  if file1['title'] == 'test.pkl':
     TESTPIK = file1['id']


In [0]:
def gDriveAuthorization(auth_name,save_dir):
  auth.authenticate_user()
  gauth = GoogleAuth()
  # Try to load saved client credentials
  authcred_dir=save_dir+"/"+auth_name+".txt"
  
  if gauth.credentials is None:
      if os.path.isfile(authcred_dir):
        gauth.LoadCredentialsFile(authcred_dir)
      # Authenticate if they're not there
      gauth.credentials = GoogleCredentials.get_application_default()
      gauth.SaveCredentialsFile(authcred_dir)
  elif gauth.access_token_expired:
      # Refresh them if expired
      gauth.Refresh()
      gauth.SaveCredentialsFile(authcred_dir)
  else:
      # Initialize the saved creds
      gauth.Authorize()
  
  return GoogleDrive(gauth)


In [0]:


label_pairs ={
    'adho_mukha_svanasana':1,
    'bharadvajasana':2,
    'pasasana':3,
    'salamba_sarvangasana':4,
    'salamba_sirsasana':5,
    'virabhadrasana_I':6,
    'virabhadrasana_II':7,
    'virabhadrasana_III':8
}

In [0]:
from tensorflow.python.tools import freeze_graph
import os
import tensorflow as tf


In [0]:

from __future__ import division, print_function, absolute_import

import tflearn

# Residual blocks
# 32 layers: n=5, 56 layers: n=9, 110 layers: n=18
n = 5

# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
#img_prep.add_featurewise_zero_center(per_channel=True)

# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
#img_aug.add_random_flip_leftright()
#img_aug.add_random_blur()
#img_aug.add_random_crop([64, 64], padding=4)

# Building Residual Network
network = tflearn.input_data(shape=[None, 64, 64, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
network = tflearn.conv_2d(network, 16, 3, regularizer='L2', weight_decay=0.0001)
network = tflearn.residual_block(network, n, 16)
network = tflearn.residual_block(network, 1, 32, downsample=True)
network = tflearn.residual_block(network, n-1, 32)
network = tflearn.residual_block(network, 1, 64, downsample=True)
network = tflearn.residual_block(network, n-1, 64)
network = tflearn.batch_normalization(network)
network = tflearn.activation(network, 'relu')
network = tflearn.global_avg_pool(network)
# Regression
network = tflearn.fully_connected(network, 9, activation='softmax')
mom = tflearn.Momentum(0.1, lr_decay=0.1, decay_step=32000, staircase=True)
network = tflearn.regression(network,optimizer=mom,loss='categorical_crossentropy')


In [0]:
drive=gDriveAuthorization("skeletal-insights-creds",".")
model_name = "resnet32_skeletal_insights"
# Auto-iterate through all files in the root folder.
model_parentfolder_id="1Ay8W74_15gHTpEz-SARe-mxXBE-yTKoe"
file_list = drive.ListFile({'q': "'{parent_id}'".format(parent_id=model_parentfolder_id) + " in parents and trashed=false"}).GetList()
for file1 in file_list:
  if file1['title'] == model_name :
   model_dir = drive.ListFile({'q': "'{folderid}' in parents and trashed=false".format(folderid=file1['id'])}).GetList()
for file1 in file_list:
   for file2 in model_dir:
      drive.CreateFile({'id': file2['id']}).GetContentFile(file2['title']) 

In [0]:
def authenticateWithGDrive():
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  return GoogleDrive(gauth)
#Recursive function to walk through google drive repo        
def retrieveFolderfromGoogleDrive(parentfolder_id,targetdir='.') :
  drive=authenticateWithGDrive()
  file_list = drive.ListFile({'q': "'{parent_id}'".format(parent_id=parentfolder_id) + " in parents and trashed=false"}).GetList()
  for file1 in file_list:
    #print(file1['title'] +'__'+file1['mimeType'])
    sub_dir1=targetdir+'/'+file1['title']
    #if folder call same function
    if '.git' not in file1['title'] :
      if file1['mimeType'] == "application/vnd.google-apps.folder":
          if not os.path.isdir(sub_dir1):
            !mkdir {sub_dir1}
          retrieveFolderfromGoogleDrive(file1['id'],sub_dir1)
      else  :
        if not os.path.isfile(sub_dir1):
          drive.CreateFile({'id': file1['id']}).GetContentFile(sub_dir1)
dependencies_folder_id='19tYr3vwQsxc3l6HQLvfIgSK5gfr23soR'
retrieveFolderfromGoogleDrive(dependencies_folder_id)

In [0]:
model = tflearn.DNN(network)

In [0]:
model.load(model_name)


In [0]:
del tf.get_collection_ref(tf.GraphKeys.TRAIN_OPS)[:]
model.save(model_name)

In [0]:
import os, argparse

# The original freeze_graph function
# from tensorflow.python.tools.freeze_graph import freeze_graph 

# dir = os.path.dirname(os.path.realpath(__file__))

def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert 
    all its variables into constant 
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names, 
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)
        
        # get graph definition
        gd = sess.graph.as_graph_def()

        # fix batch norm nodes
        for node in gd.node:
          if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in range(len(node.input)):
              if 'moving_' in node.input[index]:
                node.input[index] = node.input[index] + '/read'
          elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr: del node.attr['use_locking']
          
          if 'dilations' in node.attr:
            del node.attr['dilations'] 
            
        #Remove attr

        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            gd, # The graph_def is used to retrieve the nodes 
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        ) 

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def


In [0]:
from tensorflow.python.tools import strip_unused_lib


In [0]:
freeze_graph('.',"FullyConnected/Softmax")

In [0]:
!cp frozen_model.pb $model_name'.pb'

In [0]:
!rm resnet32_skeletal_insights.py

In [0]:
model_folder_id='1Ay8W74_15gHTpEz-SARe-mxXBE-yTKoe'

In [0]:
def uploadFileToGoogleDrive(local_file_dir,name,parentfolder_id):
  drive=gDriveAuthorization("skeletal-insights-creds",".")
  file=drive.CreateFile({'title': name,'parents':[{"kind": "drive#fileLink", "id": parentfolder_id}]})
  file.SetContentFile(local_file_dir)
  file.Upload()
  return file.get('id')

In [0]:
uploadFileToGoogleDrive(model_name+".pb",model_name+".pb",model_folder_id)

In [0]:
from pydrive.drive import GoogleDrive

drive = GoogleDrive(gauth) # Create GoogleDrive instance with authenticated GoogleAuth instance

# Auto-iterate through all files in the root folder.
file_list = drive.ListFile({'q': "'1vQhQXjy4X9woILOTXeY3C-tz14xJkBQE' in parents and trashed=false"}).GetList()
for file1 in file_list:
  if file1['title'] == 'train.pkl':
     PIK = file1['id']
     print('title: %s, id: %s' % (file1['title'], file1['id']))
  elif file1['title'] == 'validation.pkl':
     VALIDPIK = file1['id']
     print('title: %s, id: %s' % (file1['title'], file1['id']))
  elif file1['title'] == 'test.pkl':
     TESTPIK = file1['id']
     print('title: %s, id: %s' % (file1['title'], file1['id']))


In [0]:
testFile = drive.CreateFile({'id': TESTPIK}).GetContentFile('test.pkl')

In [0]:
X = []
Y = []
X_validate = []
Y_validate = []
X_test = []
Y_test = []

label_pairs ={
    'adho_mukha_svanasana':1,
    'bharadvajasana':2,
    'pasasana':3,
    'salamba_sarvangasana':4,
    'salamba_sirsasana':5,
    'virabhadrasana_I':6,
    'virabhadrasana_II':7,
    'virabhadrasana_III':8
}


In [0]:
X_type = 'float64'

with open('test.pkl', "rb") as f:
    while True :
        try:
            record=pickle.load(f)
            X_test.append(record[0].astype(X_type))
            Y_test.append(label_pairs[record[1]])
        except EOFError:
            print('processing done!')
            break
            
print ("Test Data Size :" , len(X_test))


In [0]:
def getActivations(layer,stimuli,model):
    sess=model.session
    units = sess.run(layer,feed_dict={x:np.reshape(stimuli,[1,784],order='F'),keep_prob:1.0})
    plotNNFilter(units)
    
def plotNNFilter(units):
    filters = units.shape[3]
    plt.figure(1, figsize=(20,20))
    n_columns = 6
    n_rows = math.ceil(filters / n_columns) + 1
    for i in range(filters):
        plt.subplot(n_rows, n_columns, i+1)
        plt.title('Filter ' + str(i))
        plt.imshow(units[0,:,:,i], interpolation="nearest", cmap="gray")

In [0]:
layer=tflearn.variables.get_layer_variables_by_name("GlobalAvgPool")
getActivations(layer,X_test[0],model)